# TV Script Generation


Neste projeto irei gerar meu próprio script de Os Simpsons utilizando Rede Neural Recorrente (RNN - Recurrent Neural Network). Os dados de treinamento são de 27 temporadas estão disponíveis em  [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data). Como resultado será a Rede Neural irá gerar uma cena para o episódio [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).

## Importar os dados

Os dados já estão disponiveis. Ele contem uma das cenas e Moe's Tavern

In [6]:
""" Importando os dados"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)

# Ignorando dados do epsódios que não se referem ao script
text = text[81:]

## Exploração dos dados


In [21]:
range_sentenca = (0,10)


import numpy as np

print("Estatísticas do Dataset")
print('Quantidade de palavras unicas: {}'.format(len({palavra:None for palavra in text.split()})))
cenas = text.split('\n\n')
print('Quantidade de cenas: {}'.format(len(cenas)))
qtd_sentenca_cena = [cena.count('\n') for cena in cenas]
print('Média do número de sentenças em cada cena:{}'.format(np.average(qtd_sentenca_cena)))

sentenca = [sentenca for cena in cenas for sentenca in cena.split('\n')]
print('Quantidade de sentenças (linhas): {}'.format(len(sentenca)))

qtd_palavras = [len(qtdPalavra.split()) for qtdPalavra in sentenca]
print('Quantidade média de palavra por sentença: {}'.format(np.average(qtd_palavras)))

print()

print('Sentencas de {} a {}:'.format(*range_sentenca))
print('\n'.join(text.split('\n')[range_sentenca[0]: range_sentenca[1]]))


Estatísticas do Dataset
Quantidade de palavras unicas: 11492
Quantidade de cenas: 262
Média do número de sentenças em cada cena:15.251908396946565
Quantidade de sentenças (linhas): 4258
Quantidade média de palavra por sentença: 11.50164396430249

Sentencas de 0 a 10:

Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.



## Implementar função de preprocessamento
A primeira coisa a fazer em qualquer dataset é o preprocessamento.  Implementar as funçoes de preprocessamento abaixo
- Lookup Table
- Tokenize Punctuation

### Lookup Table
Para criar um word embedding, a primeira coisa a se fazer é tranformar dados em ID, nesta função será criado dois dicionarios:
- dicionario que transformaremos de palavras para ID, chamaremos de 'vocab_to_int'
- dicionario que transformaremos de ID para palavras, chamaremos de `int_to_vocab`

- retornar esses dicionários como uma tupla  `(vocab_to_int, int_to_vocab)`

In [29]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    vocab = set(text)
    vocab_to_int = {word:index for index,word in enumerate(vocab)}
    int_to_vocab = {index:word for index,word in enumerate(vocab)}
    
    return (vocab_to_int, int_to_vocab)
    
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation

O scrip será quebrado em um array de palabras utilizando os espaços como delimitadoes. Entretando, pontuações como pontos (.) e exclamações(!) tornam a rede neural mais dificil de destinguir algumas palavras como o "bye" e "bye!".

Implementar a função `token_lookup`para retornar um diicionario que será utilizado para tokenizar as pontuações em descrições.
Segue abaixo a descrição de cada pontuação

- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

Este dicionario será utilizado para tokenizar acentos e adicionar delimitadores. Utilizando esses tokens será mais fácil da rede neural prever qual será a próxima palavra. Para os acentos, será utilizado o pipe (||) para que a rede neurão não confunda como a palavra como parte do dialogo. Ex: ; para ||Semicolon|| e não Semicolon

In [30]:
def token_lookup():
    return {'.':'||Period||',
            ',':'||Comma||',
            '"':'||Quotation_Mark||',
            ';':'||Semicolon||', 
            '!':'||Exclamation_Mark||',
            '?':'||Question_Mark||', 
            '(':'||Left_Parentheses||', 
            ')':'||Right_Parentheses||',
            '--':'||Dash||',
            '\n':'||Return||'}

tests.test_tokenize(token_lookup)

Tests Passed


## Preprocessando todos os dados e salvandos
Rodando o bloco abaixo todos os dados preprocessado serão salvos (Não será necessários carregalos posteriormente

In [31]:
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
Para recarregar todos o dados que foram salvos anteriormente basta executar o bloco abaixo

In [1]:

import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Contruindo a rede neural

Para construir a rede neural será necessário a criação das funções abaixo.

- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Verifique a versão do Tensorflow e o Acesso a GPU (Caso possua uma)

In [2]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Verificando versão do tensorflow
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Por favor utilize a versão 1.0 ou mais atual'
print('Versão Tensroflow: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('GPU não encontrada. Utilize uma GPU para treinar a Rede Neural')
else:
    print('Dispositivo padrão GPU: {}'.format(tf.test.gpu_device_name()))

Versão Tensroflow: 1.0.1


C:\Users\Jonathan\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: UserWarning: GPU não encontrada. Utilize uma GPU para treinar a Rede Neural


### Input

Implementação da função `get_inputs()`para criar os TF Placeholder para a Rede Neural. Devem ser criados os seguintes Placeholders:

- Placeholder de entrada do texto chamado 'input'
- Placeholder de saída do texto chamado 'target'
- Placeholder de taxa de aprendizado chamado 'learning rate'

Saiba mais sobre o TF.Placeholder em [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder)

A função devera retornar a seguinte tupla de placeholders (Input, Target, LearningRate)

In [3]:
def get_inputs():    
    Input = tf.placeholder(tf.int32, shape=[None, None], name = 'input')
    Target = tf.placeholder(tf.int32, shape=[None, None], name='target')
    LearningRate = tf.placeholder(tf.float32, name= 'learning_rate')
    
    return (Input, Target, LearningRate)

tests.test_get_inputs(get_inputs)

Tests Passed


### Construir e iniciar Célula Rede Neural Recorrente (RNN)
Saiba mais em [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) e [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- O tamanho da RNN deve ser definida em `rnn_size`
- Inicializar o estado da celula usando MultRNNCell `zero_state()` 
- Saiba mais em (https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state)
- Iniciar o estado inicial da celula com o nome de `initial_state` utilizando o `tf.identity()`
- Saiba mais em (https://www.tensorflow.org/api_docs/python/tf/identity)
- Retornar celula e estado inicia na tupla `(Cell, InitialState)`

In [4]:
def get_init_cell(batch_size, rnn_size):
    
    # Quantidade de Camadados
    num_layers = 2
    # Celula comun
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    #Multi celulas (2 camadas)
    cell = tf.contrib.rnn.MultiRNNCell([lstm] * num_layers)
    # Iniciando estado inicial da celula
    initial_state = cell.zero_state(batch_size, tf.float32)
    # Setando estado da celula
    initial_state = tf.identity(initial_state, name='initial_state')    
    
    return (cell, initial_state)

tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Aplicar embedding em `input_data` utilozando Tensoflow.  Retornar embedded sequence.

In [5]:
def get_embed(input_data, vocab_size, embed_dim):
    
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed

tests.test_get_embed(get_embed)

Tests Passed


### Construir RNN

A celula de RNN foi criada na função get_init_cell(). Hora de usar a celula para criar a RNN.

- Criar a RNN utilizando [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

- Definir o estado com o nome "final_state" ao final do estado utilizando [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

- Retornar a saída e o estado final da seguindo a tupla `(Outputs, FinalState)`

In [6]:
def build_rnn(cell, inputs):
    # tf.nn.dynamic_rnn retorna um tensor RNN  e o estado
    outputs, initial_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    
    initial_state = tf.identity(initial_state, name='final_state')
    
    return (outputs, initial_state)
tests.test_build_rnn(build_rnn)

Tests Passed


### Construi a Rede Neural

Utilizar a funções implementadas acima para:

- Aplicar embedding em `input_data` usando a função `get_embed(input_data, vocab_sizem embed_dim)`

- Criar RNN utilizando `cell` e a função `build_rnn(cell, inputs)`

- Aplicar uma camada totalmente conectada (fully connected layer) com ativação linear e `vocab_size` com numero de outputs

- Retornar os logits e estado final como tupla (Logits, Final_State)

In [7]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    embedding = get_embed(input_data, vocab_size, embed_dim)
    (output, final_state) = build_rnn(cell, embedding)
    logits = tf.layers.dense(output, vocab_size, activation=None, use_bias=True)
    return (logits, final_state)

tests.test_build_nn(build_nn)

Tests Passed


### Batch

Implementar `get_batches` para criar batches do input e target utilizando `int_text`

Os batches devem sser um array de numpy com o shape `(numero de batch, 2 , tamanho do batch, tamanho da sequencia)`. Cada batch contem dois elementos:

- O primeiro elemento é um unico batch do **input** com o shape `[tamanho do batch, tamanho da sequencia]`

- O segundo elemento é um unio batch do **target** com o shape `[tamanho do batch, tamanho da sequencia]`

Se não for possivel preencher  o ultimo batch com os dados sificientes apagar o ultimo batch.

Deverá retornar um array de Numpy conforme abaixo

```
[
  # Primeiro Batch
  [
    # Batch do Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch do targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Segundo Batch
  [
    # Batch dk Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch do targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Terceiro Batch
  [
    # Batch do Input
    [[ 5  6], [11 12], [17 18]]
    # Batch do targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```
Observe que o último valor de destino no último lote é o primeiro valor de entrada do primeiro lote. Neste caso, `1`. Esta é uma técnica comum usada ao criar lotes de seqüência, embora seja bastante insensível.

In [8]:
def get_batches(int_text, batch_size, seq_length):
    
    num_batches = len(int_text) // (batch_size  * seq_length)
    
    np_text = np.array(int_text[:num_batches * (batch_size  * seq_length)])
    
    in_text = np_text.reshape(-1, seq_length)
    
    tar_text = np.roll(np_text, -1).reshape(-1, seq_length)
    output = np.zeros(shape=(num_batches, 2, batch_size, seq_length), dtype=np.int)
    
    # Prepare the output
    for idx in range(0, in_text.shape[0]):
        i = idx % num_batches
        j = idx // num_batches
        output[i,0,j,:] = in_text[idx,:]
        output[i,1,j,:] = tar_text[idx,:]
    return output

tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Definindo Hyperparametros

- Definir `num_epochs` para numero de epochs
- Definir `batch_size` para tamanho do bacth.
- Definir `rnn_size` para o tamanho das RNN.
- Definir `embed_dim` para tamanho do Embedding.
- Definir `seq_length` para tamnho da sequencia.
- Definir `learning_rate` para taxa de aprendizagem.
- Definir `show_every_n_batches` para printar o progresso de cada batch em treinamento

In [9]:

num_epochs = 100
batch_size = 256
rnn_size = 512
embed_dim = 256
seq_length = 16
learning_rate = 0.01
show_every_n_batches = 256

save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [10]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [11]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/16   train_loss = 8.823
Epoch  16 Batch    0/16   train_loss = 3.029
Epoch  32 Batch    0/16   train_loss = 0.958
Epoch  48 Batch    0/16   train_loss = 0.311
Epoch  64 Batch    0/16   train_loss = 0.204
Epoch  80 Batch    0/16   train_loss = 0.198
Epoch  96 Batch    0/16   train_loss = 0.196
Model Trained and Saved


## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [12]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Pegar Tensores
Pegar tensores de `loaded_graph` utilizando a função [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).



Pegar os tensores utilizando os nomes :
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Retorne os tensores utilizando a tupla `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [18]:
def get_tensors(loaded_graph):
    # TODO: Implement Function
    input_input = loaded_graph.get_tensor_by_name('input:0')
    initial_state = loaded_graph.get_tensor_by_name('initial_state:0')
    final_state = loaded_graph.get_tensor_by_name('final_state:0')
    probs = loaded_graph.get_tensor_by_name('probs:0')
    
    
    return (input_input, initial_state, final_state, probs )
tests.test_get_tensors(get_tensors)

Tests Passed


### Escolher a melhor palavra
Implementar função `pick_word` para escolher qual a próxima palavra a ser utilizada na sentença. Utiliznado `probabilities`

In [20]:
def pick_word(probabilities, int_to_vocab):
    pos_word = np.argmax(probabilities)
    next_word= int_to_vocab[pos_word]
    return next_word
tests.test_pick_word(pick_word)

Tests Passed


## Gerar Script

Este bloco irá gerar o script. Definir ´gen_length´ para definir o tamanho do script que será definido

In [23]:
gen_length = 200

# homer_simpson, moe_szyslak, ou Barney_Gumble | Pessoas 
prime_word = 'moe_szyslak'

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Carregar o modelo salvo
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Pegar os tensores do modelo carregado
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Configuração da eração das sentenças
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Geração das sentenças
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Prever a melhor próxima palavra
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remover os tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak:(looking for approval) heh? heh?
moe_szyslak: now, let dr..
moe_szyslak: hey, come on, there's sexy bald like...
moe_szyslak: oh, oh, what are you doing? you're gettin' some kind of beer.(got up beer) and now, get me to the big cock fight...
moe_szyslak:(annoyed) are you all lookin' at?
moe_szyslak: you're too late, homer.
homer_simpson: yeah, look, i just ordered my own principal, you won't be so...
homer_simpson: hey, we all know what i got a job here for your" best friend...
moe_szyslak: yeah, let's go.
moe_szyslak: oh, oh, what are you doing? you're gettin' some kind of beer.(got up beer) and now, get me to the big cock fight...
moe_szyslak:(annoyed) are you all lookin' at?
moe_szyslak: you're too late, homer.
homer_simpson: yeah, look, i just ordered my own


# O script gerado está sem sentido ?
Está tudo bem se o script de TV não faz sentido. Nós treinamos em menos de um megabyte de texto. Para obter bons resultados, você terá que usar um vocabulário menor ou obter mais dados. Felizmente, há mais dados! Como mencionamos na importação deste projeto, este é um subconjunto de outro conjunto de dados. Não tínhamos treinado em todos os dados, porque isso levaria muito tempo. No entanto, você é livre para treinar sua rede neural em todos os dados. Depois de concluir o projeto, é claro.
# Submeter projeto
Quando submeter o projeto, certifique-se de ter rodado todas a celulas antes de salvar o notebook. Salve o notebook como 

When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" e salve ele como arquivo HTML abaixo: "File  -> "Download as". Incluir o arquivo "helper.py" e "problem_unittests.py" na submissção.